<a href="https://colab.research.google.com/github/KotkaZ/journey-to-zero/blob/master/randomforestregressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset preprocessing



In [3]:
import numpy as nb
import pandas as pd

from sklearn import preprocessing
from sklearn import decomposition
from sklearn.preprocessing import MinMaxScaler

from matplotlib import pyplot as plt

%matplotlib inline

import datetime

### Timestamp extraction

Because crazy things happened in the past year,  we validated that, some specific dates had significantly higher electricity prices. Therefore we do weekday, month, and time extraction from the timestamp.  



In [4]:
def extract_weekday(dataset):
    splits = dataset['date'].astype(str).str.split('-')
    dataset['weekday'] = [datetime.date(int(year), int(month), int(day)).weekday() for (year, month, day) in splits]

In [5]:
def extract_month(dataset):
    dataset['month'] = [month for (_, month, _) in dataset['date'].astype(str).str.split('-')]

In [6]:
def extract_datetime(dataset):
    dataset.loc[:,'time'] = pd.to_datetime(dataset.loc[:,'time'], format="%Y-%m-%d %H:%M:%S", utc=True)
    dataset['date'] = dataset['time'].dt.date
    dataset['hour'] = dataset['time'].dt.hour

In [7]:
def one_hot_encode(dataset, columns, encoder = None) -> preprocessing.OneHotEncoder:
    if encoder:
        transformed = encoder.transform(dataset[columns])
    else:
        encoder = preprocessing.OneHotEncoder(sparse= False)
        transformed = encoder.fit_transform(dataset[columns])

    new_columns = []
    for i, column in enumerate(encoder.feature_names_in_):
        new_columns.extend([column + str(cat) for cat in encoder.categories_[i]])

    encoder_df = pd.DataFrame(transformed, index=dataset.index)
    dataset[new_columns] = encoder_df
    dataset.drop(columns=columns, inplace=True)
    return encoder

In [8]:
def extract_features(dataset):
    extract_datetime(dataset)
    extract_month(dataset)
    extract_weekday(dataset)


### Feature dropping

In Estonia, there are approximately 500\-800 millimeters of rain on average. Our dataset consisted of only about 140mm of rain, which is definitely not correct. Also, the amount of snow was inappropriate for the  
 same reason.


In [9]:
def drop_features(dataset):
    dataset.drop(columns=['snow','prcp','time','date'], inplace=True)


In [10]:
def drop_rows(dataset):
    # Deal with NaN values
    initial_len = len(dataset)
    dataset.dropna(inplace=True)
    new_len = len(dataset)
    if (initial_len != new_len):
        print(f'Dropped {initial_len - new_len} row')

    # Deal with outliners
    dataset.drop(dataset[dataset['el_price'] > 1].index , inplace=True)

In [11]:
def normalize(dataset, scaler = None) -> (pd.DataFrame, preprocessing.MinMaxScaler):
    if scaler:
        dataset_scaled = scaler.transform(dataset)
        return (dataset_scaled, scaler)
    scaler = preprocessing.MinMaxScaler()
    dataset_scaled = scaler.fit_transform(dataset)
    return (dataset_scaled, scaler)

In [12]:
def reduce_dimensions(dataset, pca = None) -> (pd.DataFrame, decomposition.PCA):
    if pca:
        dataset_reduced = pca.transform(dataset)
        return (dataset_reduced, pca)
    pca = decomposition.PCA(n_components=0.9)
    dataset_reduced = pca.fit_transform(dataset)
    return (dataset_reduced, pca)

In [13]:
def preprocess(dataset, encoder=None) -> preprocessing.OneHotEncoder:
    extract_features(dataset)
    drop_features(dataset)
    encoder = one_hot_encode(dataset, ['coco', 'weekday'], encoder)
    drop_rows(dataset)
    return encoder


### Import dataset

Here we import dataset, do inital processing and split into train and validation.

In [14]:
def read_dataset(file_name) -> pd.DataFrame:
    return pd.read_csv(file_name)

In [15]:
def extract_labels(dataset) -> (pd.DataFrame, pd.Series):
    X_train = dataset.loc[:, ~dataset.columns.isin(['consumption'])]
    y_train = dataset['consumption']
    return (X_train, y_train)

In [16]:
train_df = read_dataset('train.csv')
encoder = preprocess(train_df)


X_train, y_train = extract_labels(train_df)

X_train_norm, scaler = normalize(X_train)
X_train_reduced, pca = reduce_dimensions(X_train_norm)


Dropped 2 row


In [17]:
X_train_norm.shape

(8588, 41)

In [18]:
X_test = read_dataset('test.csv')
preprocess(X_test, encoder)

X_test_norm, _ = normalize(X_test, scaler)
print(X_test_norm.shape)
X_test_reduced, _ = reduce_dimensions(X_test_norm, pca)

(168, 41)


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train_reduced, y_train, test_size=0.2)

In [20]:
X_train.shape

(6870, 16)

In [21]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()



In [23]:
rfr.fit(X_train, y_train)

RandomForestRegressor()

In [25]:
prediction = rfr.predict(X_test_reduced)

In [26]:
prediction

array([0.50771, 0.65339, 0.41893, 0.48978, 0.41294, 0.45628, 0.39691,
       0.61722, 0.68831, 1.03386, 1.01606, 0.66181, 0.65744, 0.69063,
       0.65877, 0.71961, 0.73964, 0.88473, 1.49832, 1.09794, 1.05706,
       0.7959 , 1.72993, 1.59743, 0.62846, 0.52486, 0.53621, 0.62224,
       0.53097, 0.58486, 0.41625, 0.86283, 0.81902, 0.80769, 0.7154 ,
       0.82138, 0.79087, 1.16947, 0.83794, 1.76661, 0.88198, 0.9021 ,
       0.89835, 0.99173, 0.70756, 0.58517, 0.61365, 0.93202, 0.62879,
       0.42571, 0.30987, 0.86371, 0.85447, 0.65408, 0.64987, 0.69793,
       0.68   , 0.78106, 1.14364, 1.43276, 0.74339, 0.62557, 0.58766,
       0.76269, 0.72075, 0.59827, 0.70668, 0.7154 , 0.71394, 0.67391,
       0.91928, 0.41302, 0.5728 , 0.35619, 0.35187, 0.57156, 0.66649,
       0.59836, 0.42034, 0.52875, 0.65579, 0.54565, 0.56462, 0.58423,
       0.56795, 0.76645, 0.82933, 1.27408, 0.75102, 0.61025, 0.50069,
       0.55077, 0.54626, 0.49972, 0.37988, 0.4189 , 0.40167, 0.43898,
       0.37935, 0.55

In [27]:
X_test = read_dataset('test.csv')

In [29]:
predictions_dict = {'time':X_test.time,'consumption':prediction}
pred_df = pd.DataFrame(predictions_dict)
pred_df.to_csv('submission_RFR_V1.csv',index=False)